# Imports

In [ ]:
import numpy as np
import pandas as pd
import math
import ast
import matplotlib.pyplot as plt

# Read simulation data

In [ ]:
# After the random walk, we stored all the collected data in a csv structure
df_original = pd.read_csv('./outputs/csv_random_walk.csv')

In [ ]:
df_original['fly_position'] = df_original['fly_position'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['speed_integration'] = df_original['speed_integration'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['action_drive'] = df_original['action_drive'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['velocity'] = df_original['velocity'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['heading'] = df_original['heading'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['swing_phase'] = df_original['swing_phase'].apply(lambda x: [int(i) for i in ast.literal_eval(x)])

df_original['position_hist_LF'] = df_original['position_hist_LF'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['position_hist_LM'] = df_original['position_hist_LM'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['position_hist_LH'] = df_original['position_hist_LH'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['position_hist_RF'] = df_original['position_hist_RF'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['position_hist_RM'] = df_original['position_hist_RM'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['position_hist_RH'] = df_original['position_hist_RH'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

df_original['contact_forces_hist_LF'] = df_original['contact_forces_hist_LF'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['contact_forces_hist_LM'] = df_original['contact_forces_hist_LM'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['contact_forces_hist_LH'] = df_original['contact_forces_hist_LH'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['contact_forces_hist_RF'] = df_original['contact_forces_hist_RF'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['contact_forces_hist_RM'] = df_original['contact_forces_hist_RM'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
df_original['contact_forces_hist_RH'] = df_original['contact_forces_hist_RH'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))


#### Creat data columns for each leg

In [ ]:
# Creat a column for each leg conatining swing/stance phase information
df_original = df_original.assign(
    swing_LF=lambda x: x['swing_phase'].apply(lambda pos: pos[0]),
    swing_LM=lambda x: x['swing_phase'].apply(lambda pos: pos[1]),
    swing_LH=lambda x: x['swing_phase'].apply(lambda pos: pos[2]),
    swing_RF=lambda x: x['swing_phase'].apply(lambda pos: pos[3]),
    swing_RM=lambda x: x['swing_phase'].apply(lambda pos: pos[4]),
    swing_RH=lambda x: x['swing_phase'].apply(lambda pos: pos[5])
)

# Creat a column for each leg conatining leg position information
df_original['position_hist_LF'] = df_original['position_hist_LF'].apply(lambda pos: pos[:2])
df_original['position_hist_LM'] = df_original['position_hist_LM'].apply(lambda pos: pos[:2])
df_original['position_hist_LH'] = df_original['position_hist_LH'].apply(lambda pos: pos[:2])
df_original['position_hist_RF'] = df_original['position_hist_RF'].apply(lambda pos: pos[:2])
df_original['position_hist_RM'] = df_original['position_hist_RM'].apply(lambda pos: pos[:2])
df_original['position_hist_RH'] = df_original['position_hist_RH'].apply(lambda pos: pos[:2])

#### Devide data set between the different run of the simulation

In [ ]:
df1 = df_original[:5000]

#### Visually check simulation validity

In [ ]:
# Fly position (x,y) coordinates
x1, y1 = zip(*df1['fly_position'].apply(lambda pos: pos[:2]))

# Plot
plt.plot(x1, y1, label="Simulation 1")
#plt.plot(x2, y2, label="Simulation 2")
plt.legend()
plt.show()

# Useful functions

In [ ]:
# Define function allowing to sum only a part of element of the window
def custom_sum(x, group_size):
    return np.sum(x[::group_size])

# Define function allowing to average only a part of element of the window
def custom_mean(x, group_size):
    return np.nanmean(x[::group_size])

# Define function to compute angle between two vectors in degree or radian
def angle_between_vectors(previous_heading, new_heading, degree=True):
    cross_product = np.cross(new_heading, previous_heading)
    # Compute the angle between A and B
    angle = np.arctan2(cross_product, np.dot(new_heading, previous_heading))
    # Compute the angle in degree if needed 
    if degree:
        angle = np.degrees(angle)

    return angle

# Define function to compute angle for dataframe rows
def angle_positive(row):
    # Extract individual vectors from the row
    heading_x = row['heading'][0]
    heading_y = row['heading'][1]

    # Calculate the positive angle
    return angle_between_vectors([0, 1], [heading_x, heading_y])

# Define function to compute leg positions in the fly's bodyframe
def translate_and_rotate_to_bodyframe(x_original_coords, y_original_coords, x_translation_vector, y_translation_vector, rotation_angle):
    # Initialize empty arrays for results
    x_double_prime = np.empty_like(x_original_coords)
    y_double_prime = np.empty_like(y_original_coords)

    # Apply translation and rotation in a loop
    for i in range(len(x_original_coords)):
        # Apply translation
        x_prime = x_original_coords[i] - x_translation_vector[i]
        y_prime = y_original_coords[i] - y_translation_vector[i]
        # Apply rotation
        x_double_prime[i] = x_prime * math.cos(rotation_angle[i]) - y_prime * math.sin(rotation_angle[i])
        y_double_prime[i] = x_prime * math.sin(rotation_angle[i]) + y_prime * math.cos(rotation_angle[i])

    # Return the new coordinates after translation and rotation
    return x_double_prime, y_double_prime

# Pre-process raw data for our task

#### Useful variables

In [ ]:
# Processed data frame
df_drive = df1
df_drive = df_drive.reset_index(drop=True)

# Define the window size (w)
w = 4000
group_size = 10

#### Bodyframe legs' position transformation

In [ ]:
# Apply the angle_positive function to each row
df_drive['angle_positive'] = df_drive.apply(angle_positive, axis=1)

# Left front leg position frame transformation
LF_body_x, LF_body_y = translate_and_rotate_to_bodyframe(df_drive['position_hist_LF'].apply(lambda pos: pos[0]),
                                               df_drive['position_hist_LF'].apply(lambda pos: pos[1]),
                                               df_drive['fly_position'].apply(lambda pos: pos[0]),
                                               df_drive['fly_position'].apply(lambda pos: pos[1]),
                                               df_drive['angle_positive'])

# Right front leg position frame transformation
RF_body_x, RF_body_y = translate_and_rotate_to_bodyframe(df_drive['position_hist_RF'].apply(lambda pos: pos[0]),
                                               df_drive['position_hist_RF'].apply(lambda pos: pos[1]),
                                               df_drive['fly_position'].apply(lambda pos: pos[0]),
                                               df_drive['fly_position'].apply(lambda pos: pos[1]),
                                               df_drive['angle_positive'])

# Left middle leg position frame transformation
LM_body_x, LM_body_y = translate_and_rotate_to_bodyframe(df_drive['position_hist_LM'].apply(lambda pos: pos[0]),
                                               df_drive['position_hist_LM'].apply(lambda pos: pos[1]),
                                               df_drive['fly_position'].apply(lambda pos: pos[0]),
                                               df_drive['fly_position'].apply(lambda pos: pos[1]),
                                               df_drive['angle_positive'])

# Right middle leg position frame transformation
RM_body_x, RM_body_y = translate_and_rotate_to_bodyframe(df_drive['position_hist_RM'].apply(lambda pos: pos[0]),
                                               df_drive['position_hist_RM'].apply(lambda pos: pos[1]),
                                               df_drive['fly_position'].apply(lambda pos: pos[0]),
                                               df_drive['fly_position'].apply(lambda pos: pos[1]),
                                               df_drive['angle_positive'])

# Left hind leg position frame transformation
LH_body_x, LH_body_y = translate_and_rotate_to_bodyframe(df_drive['position_hist_LH'].apply(lambda pos: pos[0]),
                                               df_drive['position_hist_LH'].apply(lambda pos: pos[1]),
                                               df_drive['fly_position'].apply(lambda pos: pos[0]),
                                               df_drive['fly_position'].apply(lambda pos: pos[1]),
                                               df_drive['angle_positive'])

# Right hind leg position frame transformation
RH_body_x, RH_body_y = translate_and_rotate_to_bodyframe(df_drive['position_hist_RH'].apply(lambda pos: pos[0]),
                                               df_drive['position_hist_RH'].apply(lambda pos: pos[1]),
                                               df_drive['fly_position'].apply(lambda pos: pos[0]),
                                               df_drive['fly_position'].apply(lambda pos: pos[1]),
                                               df_drive['angle_positive'])

# Creat columns with leg position in the body frame for each leg
df_drive['LF_body'] = np.array([LF_body_x, LF_body_y]).T.tolist()
df_drive['RF_body'] = np.array([RF_body_x, RF_body_y]).T.tolist()
df_drive['LM_body'] = np.array([LM_body_x, LM_body_y]).T.tolist()
df_drive['RM_body'] = np.array([RM_body_x, RM_body_y]).T.tolist()
df_drive['LH_body'] = np.array([LH_body_x, LH_body_y]).T.tolist()
df_drive['RH_body'] = np.array([RH_body_x, RH_body_y]).T.tolist()

# Display the new dataframe
display(df_drive)

#### Stride length (along longitudinal axis)

In [ ]:
# Tripod A (LF - RM - LH) stride length
# Creat full None "stride length" columns with the correct length
df_drive['stride_length_LF'] = pd.Series([None] * len(df_drive))
df_drive['stride_length_RM'] = pd.Series([None] * len(df_drive))
df_drive['stride_length_LH'] = pd.Series([None] * len(df_drive))

# Extract indices of swing phase starts
swing_stance_indices_LF = df_drive.index[df_drive['swing_LF'].diff() == 1].tolist()
swing_stance_indices_RM = df_drive.index[df_drive['swing_RM'].diff() == 1].tolist()
swing_stance_indices_LH = df_drive.index[df_drive['swing_LH'].diff() == 1].tolist()

# Complete the "stride length" columns at previously extracted swing indices with leg position (in bodyframe)
df_drive.loc[swing_stance_indices_LF, 'stride_length_LF'] = df_drive.loc[swing_stance_indices_LF, 'LF_body']
df_drive.loc[swing_stance_indices_RM, 'stride_length_RM'] = df_drive.loc[swing_stance_indices_RM, 'RM_body']
df_drive.loc[swing_stance_indices_LH, 'stride_length_LH'] = df_drive.loc[swing_stance_indices_LH, 'LH_body']

# Tripod B (RF - LM - RH) stride length
# Creat full None "stride length" columns with the correct length
df_drive['stride_length_RF'] = pd.Series([None] * len(df_drive))
df_drive['stride_length_LM'] = pd.Series([None] * len(df_drive))
df_drive['stride_length_RH'] = pd.Series([None] * len(df_drive))

# Extract indices of swing phase starts
swing_stance_indices_RF = df_drive.index[df_drive['swing_RF'].diff() == 1].tolist()
swing_stance_indices_LM = df_drive.index[df_drive['swing_LM'].diff() == 1].tolist()
swing_stance_indices_RH = df_drive.index[df_drive['swing_RH'].diff() == 1].tolist()

# Complete the "stride length" columns at previously extracted swing indices with leg position (in bodyframe)
df_drive.loc[swing_stance_indices_RF, 'stride_length_RF'] = df_drive.loc[swing_stance_indices_RF, 'RF_body']
df_drive.loc[swing_stance_indices_LM, 'stride_length_LM'] = df_drive.loc[swing_stance_indices_LM, 'LM_body']
df_drive.loc[swing_stance_indices_RH, 'stride_length_RH'] = df_drive.loc[swing_stance_indices_RH, 'RH_body']

# Create a new column 'euclidean_distance'
indices_list = [swing_stance_indices_LF, swing_stance_indices_RM, swing_stance_indices_LH,
                swing_stance_indices_RF, swing_stance_indices_LM, swing_stance_indices_RH]
legs_list = ["LF", "RM", "LH", "RF", "LM", "RH"]

# Iterate for ech legs over the start swing indices to compute stride length (along longitudinal axis)
for leg, indices in zip(legs_list, indices_list):
    # Creat a new column for each leg
    df_drive[f'y_diff_{leg}'] = None 
    # Iterate over indices
    for i in range(1, len(indices)-1):
        # get final stance position indices
        current_index = indices[i]
        # get initial stance position indices
        previous_index = indices[i - 1]
        # get final stance leg postion
        current_array = df_drive.at[current_index, f'stride_length_{leg}']
        # get initial stance leg postion
        previous_array = df_drive.at[previous_index, f'stride_length_{leg}']
        # Compute and store stride length
        df_drive.at[current_index, f'y_diff_{leg}'] = np.abs(current_array[1] - previous_array[1])

#### Drive difference

In [ ]:
# For each time step compute drive difference (Delta L - Delta R)
df_drive["drive_diff"] = df_drive['action_drive'].apply(lambda pos: pos[0]) - df_drive['action_drive'].apply(lambda pos: pos[1])

#### Sliding window: grouped features computation

In [ ]:
# Create w "artificial" rows at the beginning to start rolling window method
df_temp = pd.DataFrame({'LF_body': [[0, 0]],
                        'LM_body': [[0, 0]],
                        'LH_body': [[0, 0]],
                        'RF_body': [[0, 0]],
                        'RM_body': [[0, 0]],
                        'RH_body': [[0, 0]],
                        'action_drive': [np.array([0, 0])],
                        'velocity': [np.array([np.NaN, np.NaN, np.NaN])]})


# Duplicate the DataFrame to form w rows
df_temp_duplicated = pd.concat([df_temp] * w, ignore_index=True)
df_drive = pd.concat([df_temp_duplicated, df_drive], ignore_index=True)

# Create columns with each drive sum over the sliding window
df_drive['d1_sum'] = df_drive['action_drive'].apply(lambda x: x[0]).rolling(window=w, min_periods=1).agg(lambda x: custom_sum(x, group_size))
df_drive['d2_sum'] = df_drive['action_drive'].apply(lambda x: x[1]).rolling(window=w, min_periods=1).agg(lambda x: custom_sum(x, group_size))

# Create columns with each drive sum over the sliding window
df_drive["drive_diff_mean"] = df_drive["drive_diff"].rolling(window=w, min_periods=1).agg(lambda x: custom_mean(x, group_size))

# Create columns with leg mean y position in the bodyframe over the sliding window
df_drive['LF_y_mean'] = df_drive['LF_body'].apply(lambda x: x[1]).rolling(window=w, min_periods=1).apply(lambda x : np.nanmean(x))
df_drive['LM_y_mean'] = df_drive['LM_body'].apply(lambda x: x[1]).rolling(window=w, min_periods=1).apply(lambda x : np.nanmean(x))
df_drive['LH_y_mean'] = df_drive['LH_body'].apply(lambda x: x[1]).rolling(window=w, min_periods=1).apply(lambda x : np.nanmean(x))
df_drive['RF_y_mean'] = df_drive['RF_body'].apply(lambda x: x[1]).rolling(window=w, min_periods=1).apply(lambda x : np.nanmean(x))
df_drive['RM_y_mean'] = df_drive['RM_body'].apply(lambda x: x[1]).rolling(window=w, min_periods=1).apply(lambda x : np.nanmean(x))
df_drive['RH_y_mean'] = df_drive['RH_body'].apply(lambda x: x[1]).rolling(window=w, min_periods=1).apply(lambda x : np.nanmean(x))

# Create columns with mean stride length over the sliding window
df_drive['y_diff_LF_mean'] = df_drive['y_diff_LF'].rolling(window=w, min_periods=1).apply(lambda x : np.nanmean(x))
df_drive['y_diff_LM_mean'] = df_drive['y_diff_LM'].rolling(window=w, min_periods=1).apply(lambda x : np.nanmean(x))
df_drive['y_diff_LH_mean'] = df_drive['y_diff_LH'].rolling(window=w, min_periods=1).apply(lambda x : np.nanmean(x))
df_drive['y_diff_RF_mean'] = df_drive['y_diff_RF'].rolling(window=w, min_periods=1).apply(lambda x : np.nanmean(x))
df_drive['y_diff_RM_mean'] = df_drive['y_diff_RM'].rolling(window=w, min_periods=1).apply(lambda x : np.nanmean(x))
df_drive['y_diff_RH_mean'] = df_drive['y_diff_RH'].rolling(window=w, min_periods=1).apply(lambda x : np.nanmean(x))

# Create columns with mean velocity over the sliding window
df_drive['v1_mean'] = df_drive['velocity'].apply(lambda x: x[0]).rolling(window=w, min_periods=1).agg(lambda x: custom_mean(x, 1))
df_drive['v2_mean'] = df_drive['velocity'].apply(lambda x: x[1]).rolling(window=w, min_periods=1).agg(lambda x: custom_mean(x, 1))

# Create columns with mean velocity shifted by the window size to be able to compute heading change
df_drive['mean_v1_minus_w'] = df_drive['v1_mean'].shift(w, fill_value=0)
df_drive['mean_v2_minus_w'] = df_drive['v2_mean'].shift(w, fill_value=0)

# Drop first rows (2*w) as we do not have enought information yto compute heading change
df_drive = df_drive.iloc[(2*w):].reset_index(drop=True)

# Compute headong change angle using mean velocity vectors
df_drive['angle_degrees'] = df_drive.apply(lambda row: angle_between_vectors([row['v1_mean'], row['v2_mean']],
                                                          [row['mean_v1_minus_w'], row['mean_v2_minus_w']], True), axis=1)

#### Save processed dataframe in a pickle

In [ ]:
df_drive.to_pickle('preprocessed_final.pkl')

#### Create a loop to save all dataframe of a given simulation

In [ ]:
dfs = []

# Load and concatenate pickled DataFrames
for i in range(1, 9):
    file_path = f'Simulation_1_run_{i}.pkl'
    df = pd.read_pickle(file_path)
    dfs.append(df)

for i in range(1, 3):
    file_path = f'Simulation_2_run_{i}.pkl'
    df = pd.read_pickle(file_path)
    dfs.append(df)

# Concatenate DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged DataFrame to a pickle file
merged_df.to_pickle('preprocessed_data_w_4000.pkl')